# Explaining Model Decisions and Performance

## Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [1]:
import pandas as pd
from pmlb import fetch_data

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import (
    plot_anomalies,
    plot_dataset,
    plot_feature_importances,
)

# Section 1: Train and Configure the Trainee

## Step 1: Load Data

We are using the Adult dataset in this recipe.

In [2]:
# Load adult data
df = fetch_data('adult', local_cache_dir="../../../data/adult")

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education',
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain',
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,36.0,4,113397.0,11,9.0,2,6,0,4,1,0.0,0.0,40.0,39,1
1,35.0,7,225385.0,11,9.0,0,10,4,4,0,0.0,0.0,40.0,39,1
2,59.0,2,120617.0,11,9.0,5,11,4,2,0,0.0,0.0,40.0,39,1
3,33.0,4,101352.0,14,15.0,4,10,1,4,0,0.0,0.0,50.0,39,0
4,61.0,4,92691.0,11,9.0,2,1,0,4,1,0.0,0.0,3.0,39,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58.0,4,35520.0,15,10.0,2,6,0,4,1,7688.0,0.0,40.0,39,0
996,60.0,0,134152.0,6,5.0,0,0,1,2,1,0.0,0.0,35.0,39,1
997,33.0,4,44559.0,11,9.0,2,10,0,4,1,0.0,0.0,35.0,39,1
998,25.0,4,199947.0,11,9.0,2,1,5,4,0,0.0,0.0,40.0,39,1


## Step 2: Define the Feature Attributes

For all Howso Engine usage, we must define feature attributes. We will also define our action and context features.

In [3]:
types = {
    "education": "nominal"
}

# Infer features attributes
features = infer_feature_attributes(df, types=types)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

## Step 3: Create the Trainee, Train, and Analyze

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features, context_features=features.get_names(without=action_features))


## Step 4: Evaluate Trainee Accuracy

In [5]:
accuracy = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["accuracy"]
        }
).to_dataframe()['target'].iloc[0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.815


# Section 2: Explain Predictions

How were the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Step 1: Inspect Feature Importance (Global)

Feature importance information provides insight into the features which are primary drivers for each prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be computed at the local level (regional model for each case).

In [6]:
robust_accuracy_contributions = t.react_aggregate(
    feature_influences_action_feature=action_features[0],
    details = {"feature_robust_accuracy_contributions": True}
)['feature_robust_accuracy_contributions']
plot_feature_importances(robust_accuracy_contributions.T, title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

MDA values for each feature indicate to the user which features have the most significant individual predictive power. Those with higher values are more important for the model's predictions. Furthermore, features with negative MDA values can even be considered detrimental to model accuracy. However, it's important to note that these are global values and features with negative MDA values could still be beneficial for model accuracy in some regions of the data.

## Step 2: Feature Uncertainty (Global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason Howso Engine can maintain its level of performance despite noisy data is through the characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. 

> Note: Feature Residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [7]:
global_feature_residuals = t.react_aggregate(
    prediction_stats_action_feature=action_features[0],
    details = {
        "prediction_stats": True,
        "selected_prediction_stats": ["mae"]
    }
).to_dataframe()
global_feature_residuals = global_feature_residuals.T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)

global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,77856.949395
capital-gain,1562.848554
capital-loss,126.146278
age,10.795815
hours-per-week,9.015834
native-country,2.855187
education-num,1.589787
occupation,0.877949
education,0.694226
relationship,0.507416


# Section 3: Inspect the Data for Anomalies

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Step 1: Identify Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. An inlier could be a fraudulent case that is “too good to be true”. 



In [8]:
# Store the familiarity conviction into each case with `react_into_features`,
# this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,36.0,4,113397.0,11,9.0,2,6,0,4,1,0.0,0.0,40.0,39,1,0.593978,0,3da08aaa-86e3-491f-bbde-9f2927ab18b8,4.820333
1,35.0,7,225385.0,11,9.0,0,10,4,4,0,0.0,0.0,40.0,39,1,1.991966,1,3da08aaa-86e3-491f-bbde-9f2927ab18b8,9.662678
2,59.0,2,120617.0,11,9.0,5,11,4,2,0,0.0,0.0,40.0,39,1,3.342784,2,3da08aaa-86e3-491f-bbde-9f2927ab18b8,11.614757
3,33.0,4,101352.0,14,15.0,4,10,1,4,0,0.0,0.0,50.0,39,0,16.918810,3,3da08aaa-86e3-491f-bbde-9f2927ab18b8,18.443925
4,61.0,4,92691.0,11,9.0,2,1,0,4,1,0.0,0.0,3.0,39,1,7.101202,4,3da08aaa-86e3-491f-bbde-9f2927ab18b8,18.449176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58.0,4,35520.0,15,10.0,2,6,0,4,1,7688.0,0.0,40.0,39,0,8.786403,995,3da08aaa-86e3-491f-bbde-9f2927ab18b8,26.679914
996,60.0,0,134152.0,6,5.0,0,0,1,2,1,0.0,0.0,35.0,39,1,16.559579,996,3da08aaa-86e3-491f-bbde-9f2927ab18b8,24.320827
997,33.0,4,44559.0,11,9.0,2,10,0,4,1,0.0,0.0,35.0,39,1,4.906418,997,3da08aaa-86e3-491f-bbde-9f2927ab18b8,12.955811
998,25.0,4,199947.0,11,9.0,2,1,5,4,0,0.0,0.0,40.0,39,1,1.229936,998,3da08aaa-86e3-491f-bbde-9f2927ab18b8,7.669012


Here we set a threshold for familarity conviction that can be used to identify cases as anomalies. In the case of this notebook, we use a threshold of 0.50. Cases with a conviction value below this threshold will be deemed anomalous. Then we split up these anomalies by the mean distance contribution with those lower distance contributions being inliers and those with larger distance contributions being outliers.

In [9]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.5

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Step 2: Inspect Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [10]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,32.0,4,203674.0,7,12.0,0,10,4,4,0,0.0,880.0,36.0,39,1,0.063296,523,3da08aaa-86e3-491f-bbde-9f2927ab18b8,204.090209,outlier
1,61.0,4,29059.0,11,9.0,0,12,4,4,0,0.0,2754.0,25.0,39,1,0.065017,723,3da08aaa-86e3-491f-bbde-9f2927ab18b8,200.073601,outlier
2,51.0,4,197163.0,14,15.0,4,10,1,4,0,0.0,2559.0,50.0,39,0,0.093525,462,3da08aaa-86e3-491f-bbde-9f2927ab18b8,152.259439,outlier
3,58.0,4,473836.0,5,4.0,6,5,2,4,0,0.0,0.0,45.0,13,1,0.103244,387,3da08aaa-86e3-491f-bbde-9f2927ab18b8,143.573767,outlier
4,52.0,4,311931.0,4,3.0,2,12,5,4,0,0.0,0.0,15.0,8,1,0.104073,345,3da08aaa-86e3-491f-bbde-9f2927ab18b8,142.819063,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,50.0,6,105010.0,11,9.0,2,5,0,4,1,0.0,2051.0,20.0,39,1,0.454972,471,3da08aaa-86e3-491f-bbde-9f2927ab18b8,60.396899,outlier
67,50.0,6,371305.0,11,9.0,2,12,0,4,1,0.0,1902.0,60.0,39,0,0.465112,108,3da08aaa-86e3-491f-bbde-9f2927ab18b8,60.660846,outlier
68,26.0,4,201579.0,15,10.0,4,3,4,4,1,0.0,0.0,40.0,26,1,0.474441,616,3da08aaa-86e3-491f-bbde-9f2927ab18b8,59.853165,outlier
69,43.0,4,50356.0,15,10.0,2,3,0,4,1,0.0,1485.0,50.0,39,1,0.483498,283,3da08aaa-86e3-491f-bbde-9f2927ab18b8,59.204025,outlier


For each outlier, we will compute some of its Feature Residual Convictions to understand which feature values are the most surprising, which will help the user understand what makes each case anomalous.

In [11]:
# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {
    'feature_full_residual_convictions_for_case': True
}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [12]:
# Extract the case feature residual convictions
case_feature_residual_convictions = pd.DataFrame(results['details']['feature_full_residual_convictions_for_case'])[df.columns.tolist()]

In [13]:
fig = plot_anomalies(outliers, case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")
fig.show(width=1750, height=750)

The heat map explains the reason why each case was an outlier. The darker the shade of red, the more surprising the feature value is, which contributes to the case's status as an outlier.

## Step 3: Inspect Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [14]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,27.0,4,190303.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.107689,647,3da08aaa-86e3-491f-bbde-9f2927ab18b8,0.142647,inlier
1,27.0,4,189777.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.108306,360,3da08aaa-86e3-491f-bbde-9f2927ab18b8,0.145811,inlier
2,28.0,4,190911.0,11,9.0,2,3,0,4,1,0.0,0.0,40.0,39,1,0.131639,628,3da08aaa-86e3-491f-bbde-9f2927ab18b8,0.289170,inlier
3,20.0,0,121023.0,15,10.0,4,0,3,4,0,0.0,0.0,40.0,39,1,0.162870,530,3da08aaa-86e3-491f-bbde-9f2927ab18b8,0.584098,inlier
4,20.0,0,124242.0,15,10.0,4,0,3,4,0,0.0,0.0,40.0,39,1,0.166810,79,3da08aaa-86e3-491f-bbde-9f2927ab18b8,0.617685,inlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,39.0,4,141029.0,11,9.0,2,1,5,4,0,0.0,0.0,40.0,39,0,0.477973,898,3da08aaa-86e3-491f-bbde-9f2927ab18b8,3.642304,inlier
58,37.0,4,99151.0,11,9.0,2,1,5,4,0,0.0,0.0,40.0,39,0,0.480398,270,3da08aaa-86e3-491f-bbde-9f2927ab18b8,3.656373,inlier
59,34.0,4,149507.0,11,9.0,2,3,0,4,1,0.0,0.0,55.0,39,0,0.497701,299,3da08aaa-86e3-491f-bbde-9f2927ab18b8,3.795798,inlier
60,40.0,2,208277.0,9,13.0,0,10,4,4,0,0.0,0.0,35.0,39,1,0.498632,643,3da08aaa-86e3-491f-bbde-9f2927ab18b8,3.839890,inlier


Similarly to what we did with the outliers, we will use the Feature Residual Convictions to understand what feature values are unusual. In this case, we may expect to see many feature values with very high convictions, indicating that the feature value is particularly unsurprising.

In [15]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices,
                  preserve_feature_values=df.columns.tolist(),
                  leave_case_out=True,
                  details=details)

In [16]:
# Extract the case feature residual convictions
case_feature_residual_convictions = pd.DataFrame(results['details']['feature_full_residual_convictions_for_case'])[df.columns.tolist()]

In [17]:
fig = plot_anomalies(inliers, case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")
fig.show(width=1500, height=750)

The heat map explains the reason why each case was an inlier. The more unspurprising the value is. If this seems unintuitive, imagine a dataset where values are always hard to predict with an error of less than 15.0. If a case were to suddenly appear where every value was predicted perfectly, its Feature Residual Conviction values would all be extremely high (dark blue in the case of this visualization).

# Section 4: Finding Potential Improvements in the Dataset

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

## Step 1: Compute Case Feature Residual Convictions

Case Feature Residual Convicion is a ratio of the local residual for each feature to the case residual. Inspecting these conviction values helps users understand what regions of their data are easier or more difficult to predict compared to the average across the dataset.

Understanding this can help users understand where more data collection may be beneficial.

In [18]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]

In [19]:
# Residual convictions are output via the case_feature_residual_convictions explanation
details = {
    'feature_full_residual_convictions_for_case':True,
    'features': ['target'],
}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(),
                     leave_case_out=True,
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(),
                     action_features=action_features,
                     details=details)

In [20]:
# Extract residual conviction
target_residual_convictions = [ x['target'] for x in new_result['details']['feature_full_residual_convictions_for_case'] ]

# Binarize residual conviction
convict_threshold = 0.50
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [21]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,36.0,4,113397.0,11,9.0,2,6,0,4,1,...,40.0,39,1,0.593978,0,3da08aaa-86e3-491f-bbde-9f2927ab18b8,4.820333,0.207455,1.160600,0
1,35.0,7,225385.0,11,9.0,0,10,4,4,0,...,40.0,39,1,1.991966,1,3da08aaa-86e3-491f-bbde-9f2927ab18b8,9.662678,0.103491,0.969059,0
2,59.0,2,120617.0,11,9.0,5,11,4,2,0,...,40.0,39,1,3.342784,2,3da08aaa-86e3-491f-bbde-9f2927ab18b8,11.614757,0.086097,0.967719,0
3,33.0,4,101352.0,14,15.0,4,10,1,4,0,...,50.0,39,0,16.918810,3,3da08aaa-86e3-491f-bbde-9f2927ab18b8,18.443925,0.054218,1.139972,0
4,61.0,4,92691.0,11,9.0,2,1,0,4,1,...,3.0,39,1,7.101202,4,3da08aaa-86e3-491f-bbde-9f2927ab18b8,18.449176,0.054203,1.508996,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,58.0,4,35520.0,15,10.0,2,6,0,4,1,...,40.0,39,0,8.786403,995,3da08aaa-86e3-491f-bbde-9f2927ab18b8,26.679914,0.037481,1.427325,0
996,60.0,0,134152.0,6,5.0,0,0,1,2,1,...,35.0,39,1,16.559579,996,3da08aaa-86e3-491f-bbde-9f2927ab18b8,24.320827,0.041117,1.190835,0
997,33.0,4,44559.0,11,9.0,2,10,0,4,1,...,35.0,39,1,4.906418,997,3da08aaa-86e3-491f-bbde-9f2927ab18b8,12.955811,0.077185,0.689248,0
998,25.0,4,199947.0,11,9.0,2,1,5,4,0,...,40.0,39,1,1.229936,998,3da08aaa-86e3-491f-bbde-9f2927ab18b8,7.669012,0.130395,1.331374,0


In [22]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()

    m = (max_size - min_size) / (max_value - min_value)

    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

With the Residual Convictions computed for the "target" feature across all the trained data, we can visualize the data to attempt to find the regions of the data where convictions are particularly low. 

In [23]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  


# Conclusion

In this recipe, we demonstrate many of the metrics that the Trainee provides which helps users understand the model's decision making process such as feature MDA and feature residuals.

Additionally, we show how different types of conviction can be used to identify inliers, outliers, and regions of the data where increased data collection or feature engineering may be appropriate.